# Morphosyntactic tagging

Morphosyntactic tagging is one of the core algorithms in NLP. It assigns morphological
and (in some languages) syntactic tags to the words in a text. E.g. this allows to distinguish
between the major grammatical categories, such as nouns and verbs.


## Tasks

1. Download [docker image](https://hub.docker.com/r/djstrong/krnnt2) of KRNNT2. It includes the following tools:
   1. Morfeusz2 - morphological dictionary
   1. Corpus2 - corpus access library
   1. Toki - tokenizer for Polish
   1. Maca - morphosyntactic analyzer
   1. KRNNT - Polish tagger

>$ docker run -it -p 9200:9200 apohllo/krnnt:0.1 python3 /home/krnnt/krnnt/krnnt_serve.py /home/krnnt/krnnt/data

2. Use the tool to tag and lemmatize the law corpus.

In [47]:
import requests
import regex

def tag_and_lemmatize(data):
    response = requests.post('http://localhost:9200', data=data)
    text = response.text

    reg = r"(?<=\n\t)\S*\t\w*"
    matches = regex.finditer(reg, text, regex.MULTILINE, regex.IGNORECASE)

    corp = []
    for matchNum, match in enumerate(matches, start=1):
        spliced = match.group().split("\t")
        combined = spliced[0].lower()+":"+spliced[1]
        corp.append(combined)

    return corp

import os

def tag_and_lemmatize_law_corpus():

    # TODO: change to "ustawy"
    directory = '../ustawa/'
    fileList = os.listdir(os.getcwd() + '/' + directory)
    corp = []
    for filename in fileList:
        with open(os.path.join(directory + filename), 'r') as file:
            infile = file.read()
            corp_ = tag_and_lemmatize(infile.encode('utf-8'))
            corp.extend(corp_)
            print('', end='.')
    return corp

# corp = tag_and_lemmatize_law_corpus()
# print(corp)

In [48]:
# with open("corpus_tag_and_lemmatized.py", 'w') as file:
#     file.write("list=")
#     file.write(corp.__str__())

In [49]:
import corpus_tag_and_lemmatized
corp = corpus_tag_and_lemmatized.list
print(corp)


['dziennik:brev', '.:interp', 'ustawa:brev', '.:interp', 'z:prep', '1993:adj', 'rok:brev', '.:interp', 'numer:brev', '129:num', ',:interp', 'pozycja:brev', '.:interp', '599:adj', 'ustawa:subst', 'z:prep', 'dzień:subst', '9:adj', 'grudzień:subst', '1993:adj', 'rok:brev', '.:interp', 'o:prep', 'zmiana:subst', 'ustawa:subst', 'o:prep', 'podatek:subst', 'od:prep', 'towar:subst', 'i:conj', 'usługa:subst', 'oraz:conj', 'o:prep', 'podatek:subst', 'akcyzowy:adj', 'artykuł:brev', '.:interp', '1:adj', '.:interp', 'w:prep', 'ustawa:subst', 'z:prep', 'dzień:subst', '8:adj', 'styczeń:subst', '1993:adj', 'rok:brev', '.:interp', 'o:prep', 'podatek:subst', 'od:prep', 'towar:subst', 'i:conj', 'usługa:subst', 'oraz:conj', 'o:prep', 'podatek:subst', 'akcyzowy:adj', '(:interp', 'dziennik:brev', '.:interp', 'ustawa:brev', '.:interp', 'numer:brev', '11:num', ',:interp', 'pozycja:brev', '.:interp', '50:adj', 'i:conj', 'numer:brev', '28:num', ',:interp', 'pozycja:brev', '.:interp', '127:adj', '):interp', 'wpr

3. Using the tagged corpus compute bigram statistic for the tokens containing:
   1. lemmatized, downcased word
   > downcasowanie załatwione w poprzednim kroku
   1. morphosyntactic **category** of the word (`subst`, `fin`, `adj`, etc.)

In [50]:
def compute_bigram_statistics(corp):
    bigrams = {}
    for i in range(0, len(corp)-1):
        t1 = corp[i]
        t2 = corp[i +1]
        if (t1, t2) not in bigrams:
            bigrams[(t1, t2)] = 1
        else:
            bigrams[(t1, t2)] += 1

    bigrams = dict(sorted(bigrams.items(), key= lambda a: a[1], reverse=True))
    return bigrams

bigrams = compute_bigram_statistics(corp)
print(list(bigrams.items())[:10])

[(('|:interp', '|:interp'), 1422), (('-:interp', '-:interp'), 604), (('.:interp', '|:interp'), 156), (('ustęp:brev', '.:interp'), 147), (('artykuł:brev', '.:interp'), 100), (('):interp', 'w:prep'), 72), (('wyraz:subst', '":interp'), 69), (('w:prep', 'artykuł:brev'), 63), (('w:prep', 'ustęp:brev'), 58), (('.:interp', '1:adj'), 57)]


4. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

In [51]:
def discard_other_than_leters(bigrams):
    bigrams_filtered = dict(filter(lambda a: a[0][0].split(":")[0].isalpha() and a[0][1].split(":")[0].isalpha(), bigrams.items()))
    return bigrams_filtered

bigrams_filtered = discard_other_than_leters(bigrams)
print(list(bigrams_filtered.items())[:10])

[(('w:prep', 'artykuł:brev'), 63), (('w:prep', 'ustęp:brev'), 58), (('się:qub', 'wyraz:subst'), 36), (('otrzymywać:fin', 'brzmienie:subst'), 29), (('zastępować:fin', 'się:qub'), 25), (('dodawać:fin', 'się:qub'), 22), (('o:prep', 'który:adj'), 22), (('który:adj', 'mowa:subst'), 22), (('mowa:subst', 'w:prep'), 22), (('określić:ppas', 'w:prep'), 21)]


5. For example: "Ala ma kota", which is tagged as:
   ```
   Ala	none
           Ala	subst:sg:nom:f	disamb
   ma	space
           mieć	fin:sg:ter:imperf	disamb
   kota	space
           kot	subst:sg:acc:m2	disamb
   .	none
           .	interp	disamb
   ```
   the algorithm should return the following bigrams: `ala:subst mieć:fin` and `mieć:fin kot:subst`.

In [52]:
corp_ala = tag_and_lemmatize('Ala ma kota.'.encode('utf-8'))
bigrams_ala = compute_bigram_statistics(corp_ala)
bigrams_filtered_ala = discard_other_than_leters(bigrams_ala)
print(bigrams_filtered_ala)

{('ala:subst', 'mieć:fin'): 1, ('mieć:fin', 'kot:subst'): 1}


 >checked

6. Compute LLR statistic for this dataset.

In [53]:
# from https://github.com/tdunning/python-llr/blob/master/llr.py
import math

def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])


def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

In [54]:
def calculate_freq_list():
    freq_list= {}
    for token in corp:
        token = str(token).lower()
        if token not in freq_list:
            freq_list[str(token).lower()] =1
        else:
            freq_list[str(token).lower()] +=1

    return dict(sorted(freq_list.items(), key= lambda a: a[1], reverse=True))

freq_list = calculate_freq_list()


def calculate_LLRs(bigrams_count, freq_list):
    words_num = sum(list(map(lambda a: a[1], freq_list.items())))
    bigrams_LLRs = {}
    for item in bigrams_count.items():
        bigram = item[0]
        a = bigram[0]
        b = bigram[1]
        count = item[1]
        a_and_b = count
        a_without_b = freq_list[a] - a_and_b
        b_without_a = freq_list[b] - a_and_b
        neither_a_or_b = words_num - (freq_list[a] + freq_list[b])
        bigrams_LLRs[bigram]=llr_2x2(a_and_b, b_without_a, a_without_b, neither_a_or_b)
    bigrams_LLRs = dict(sorted(bigrams_LLRs.items(), key= lambda a: a[1], reverse=True))
    return bigrams_LLRs


bigrams_LLRs = calculate_LLRs(bigrams_filtered, freq_list)

print(list(bigrams_LLRs.items())[:10])

[(('otrzymywać:fin', 'brzmienie:subst'), 340.69407909448125), (('w:prep', 'artykuł:brev'), 338.98990259613856), (('się:qub', 'wyraz:subst'), 276.8840999913075), (('który:adj', 'mowa:subst'), 255.73610406262992), (('zastępować:fin', 'się:qub'), 255.65962178047198), (('w:prep', 'ustęp:brev'), 237.6089289543761), (('dodawać:fin', 'się:qub'), 210.1843806718232), (('o:prep', 'który:adj'), 180.83380965786364), (('rachunek:subst', 'uprościć:ppas'), 169.37892451664896), (('przeciętny:adj', 'wynagrodzenie:subst'), 162.48688127100615)]


7. Partition the entries based on the syntactic categories of the words, i.e. all bigrams having the form of
   `w1:adj` `w2:subst` should be placed in one partition (the order of the words may not be changed).

In [55]:
def partition_bigrams(bigrams_filtered):
    partitions = {}
    partitions_size={}
    for big in bigrams_filtered.items():
        cat1 = big[0][0].split(":")[1]
        cat2 = big[0][1].split(":")[1]
        if (cat1, cat2) not in partitions:
            partitions[(cat1, cat2)] = {big}
            partitions_size[(cat1, cat2)] = 1
        else:
            partitions[(cat1, cat2)].update({big})
            partitions_size[(cat1, cat2)] += 1

    partitions_size = dict(sorted(partitions_size.items(), key= lambda a: a[1], reverse=True))
    return partitions, partitions_size

partitions, partitions_size = partition_bigrams(bigrams_filtered)
print(partitions)

{('prep', 'brev'): {(('w:prep', 'ustęp:brev'), 58), (('po:prep', 'ustęp:brev'), 6), (('w:prep', 'artykuł:brev'), 63), (('w:prep', 'punkt:brev'), 10), (('z:prep', 'artykuł:brev'), 2), (('z:prep', 'ustęp:brev'), 5), (('po:prep', 'punkt:brev'), 2)}, ('qub', 'subst'): {(('się:qub', 'stawka:subst'), 3), (('także:qub', 'podkład:subst'), 1), (('się:qub', 'średnik:subst'), 1), (('się:qub', 'wyraz:subst'), 36), (('również:qub', 'świadczenie:subst'), 1), (('by:qub', 'uzupełnienie:subst'), 1), (('się:qub', 'zdanie:subst'), 1), (('się:qub', 'liczba:subst'), 1), (('wyłącznie:qub', 'towar:subst'), 1), (('także:qub', 'usługa:subst'), 1), (('się:qub', 'minister:subst'), 1), (('wyłącznie:qub', 'czynność:subst'), 1), (('się:qub', 'przepis:subst'), 1), (('się:qub', 'przecinek:subst'), 2)}, ('fin', 'subst'): {(('przysługiwać:fin', 'dodatek:subst'), 2), (('określać:fin', 'wartość:subst'), 1), (('przekraczać:fin', 'kwota:subst'), 1), (('zacją:fin', 'drogi:subst'), 1), (('ulegać:fin', 'zwiększenie:subst'), 1

8. Select the 10 largest partitions (partitions with the larges number of entries).

In [67]:
print(list(partitions_size.items())[:10])


[(('subst', 'adj'), 254), (('prep', 'subst'), 196), (('subst', 'subst'), 187), (('subst', 'prep'), 117), (('adj', 'subst'), 110), (('conj', 'subst'), 88), (('subst', 'conj'), 77), (('subst', 'ppas'), 50), (('adj', 'conj'), 46), (('adj', 'prep'), 45)]


9. Use the computed LLR measure to select 5 bigrams for each of the largest categories.

In [80]:
n = 1
for key in partitions_size.items():
    largest_cat = dict(partitions[key[0]])
    print("Category {}: {}".format(n, key))
    i=0
    for LLR_key in bigrams_LLRs.items():
        # print(LLR_key[0])
        if LLR_key[0] in largest_cat:
            print(LLR_key)
            i+=1
        if i == 5:
            break
    if n == 10:
        break
    n+=1

Category 1: (('subst', 'adj'), 254)
(('urząd:subst', 'skarbowy:adj'), 144.69188006206554)
(('kwartał:subst', 'kalendarzowy:adj'), 112.22290397128154)
(('ubezpieczenie:subst', 'społeczny:adj'), 76.01107892372686)
(('renta:subst', 'inwalidzki:adj'), 73.34579415259367)
(('prezes:subst', 'główny:adj'), 63.60757521116993)
Category 2: (('prep', 'subst'), 196)
(('dla:prep', 'dziecko:subst'), 158.7061331956918)
(('na:prep', 'podstawa:subst'), 108.63063425408768)
(('w:prep', 'przypadek:subst'), 88.62287498212254)
(('po:prep', 'wyraz:subst'), 82.04753303080201)
(('od:prep', 'dzień:subst'), 65.66112253832455)
Category 3: (('subst', 'subst'), 187)
(('zwrot:subst', 'różnica:subst'), 156.2518246986591)
(('minister:subst', 'finanse:subst'), 125.11330801735454)
(('różnica:subst', 'podatek:subst'), 118.6050132520013)
(('sprzedaż:subst', 'towar:subst'), 99.46043658627445)
(('wartość:subst', 'sprzedaż:subst'), 94.8445471354645)
Category 4: (('subst', 'prep'), 117)
(('mowa:subst', 'w:prep'), 160.662561234

10. Using the results from the previous step answer the following questions:


   i. What types of bigrams have been found?
   >Znaleziono następujące typy bigramów:
>('subst', 'adj'),('prep', 'subst'),('subst', 'subst'), ('subst', 'prep'),('adj', 'subst')
>('conj', 'subst'),('subst', 'conj'),('subst', 'ppas'),('adj', 'conj'),('adj', 'prep')


   ii. Which of the category-pairs indicate valuable multiword expressions?
   >W mojej opinii najbardziej wartościowymi grupami wyrażeń wielowyrazowych są:
>(subst, adj)(subst, subst)(adj, subst).

    Do they have anything in common?
>W każdym z nich wstępuje (subst) rzeczownik.


   iii. Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?
   >Jednostki wielowyrazowe najlepiej typować przy wykorzystaniu podejścia hybrydowego.
>LLR mierzy nie tylko kolokacyjność ale także częstość występowania.


   iiii. Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world
      problem?
   > Rozpoznawanie kategorii morfosyntaktycznych może być szczególnie przydatne w tłumaczeniu długich fraz.
>Długie sentencje zawierają często frazy oddzielone przecinkami.
>Szczegółowa analiza relacji między między frazami może poprawić jakość parsowania, która będzie przyczyniać się do wzrostu jakości tłumaczenia.



## Hints

1. A morphosyntactic analyzer provides the possible values of morphosyntactic tags for the words.
   E.g. for Polish "ma" word it can produce the following interpretations:
   ```
    ma	space
            mieć	fin:sg:ter:imperf
            mój  	adj:sg:nom:f:pos
            mój  	adj:sg:voc:f:pos
   ```
   1. The first interpretation shows that the word can be a verb in singular, in 3rd person.
   1. The second interpretation shows that the word can be an adjective in singular, in nominative, in feminine.
   1. The third interpretation shows that the word can be an adjective in singular, in vocative, in feminine.
1. The full list of tags is available at [NKJP](http://nkjp.pl/poliqarp/help/ense2.html).
1. A morphosyntactic tagger selects one of the interpretation of a word, taking into account its context.
   It can take the interpretation from a dictionary (like KRNNT), but it can also compute it dynamically (e.g.
   [COMBO](https://github.com/360er0/COMBO) is a tagger that does not need a morphosyntactic analyzer).
1. The information provided by a tagger can be useful for many applications. You can selects words from particular
   grammatical category or you can submit the data to a downstream task such as text classification.
1. More sophisticated algorithms for multiword expressions identification, such as
   [AutoPhrase](https://github.com/shangjingbo1226/AutoPhrase) take into account more features including:
   morphosyntactic tags, expression contexts, etc. and use data from e.g. Wikipedia, to automatically identify
   high-quality multiword expressions and use them to train MWE classifiers.